In [ ]:
# Uploading all professor modules
import sys
sys.path.append('../GithubModules')
print(sys.version)

In [ ]:
import pandas as pd 
URM_df=pd.read_csv('../input/RecsysDataset/data_train.csv')
ICM_genre_df=pd.read_csv('../input/RecsysDataset/data_ICM_genre.csv')
ICM_subgenre_df=pd.read_csv('../input/RecsysDataset/data_ICM_subgenre.csv')
ICM_channel_df=pd.read_csv('../input/RecsysDataset/data_ICM_channel.csv')
ICM_event_df=pd.read_csv('../input/RecsysDataset/data_ICM_event.csv')
# Extract a list of users who will be present in the final submission
target_users = pd.read_csv('../input/RecsysDataset/data_target_users_test.csv')

In [ ]:
# Just for usability/readability + convert values from float to int 

URM_df.columns = ['userID','itemID','interaction']
ICM_channel_df.columns = ['itemID','featureID','value']
ICM_event_df.columns = ['itemID','featureID','value']
ICM_genre_df.columns = ['itemID','featureID','value']
ICM_subgenre_df.columns = ['itemID','featureID','value']
URM_df['interaction'] = URM_df['interaction'].astype(int)
ICM_channel_df['value'] = ICM_channel_df['value'].astype(int)
ICM_event_df['value'] = ICM_event_df['value'].astype(int)
ICM_genre_df['value'] = ICM_genre_df['value'].astype(int)
ICM_subgenre_df['value'] = ICM_subgenre_df['value'].astype(int)

In [ ]:
#Let's use properties of the sparse matrices
import scipy.sparse as sps
import numpy as np
from numpy import linalg as LA

URM_sparse = sps.coo_matrix((URM_df['interaction'].values,(URM_df['userID'].values,URM_df['itemID'].values)))
URM_csr = URM_sparse.tocsr()

In [ ]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_user_wise
from Evaluation.Evaluator import EvaluatorHoldout

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_csr , train_percentage = 0.80)
#URM_train, URM_validation = split_train_in_two_percentage_user_wise(URM_sparse.tocsr(), train_percentage = 0.8, verbose = False)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [ ]:
######################################################################
# HERE IMPORT, DEFINE AND FIT THE THREE RECOMMENDERS YOU WANT TO HYBRID
######################################################################

In [ ]:
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        
        
        
    def fit(self, norm, alpha = 0.25, beta = 0.25):

        self.alpha = alpha
        self.beta = beta
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_3 == 0:
            raise ValueError("Norm {} of item weights for recommender 3 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * self.beta + item_weights_3 / norm_item_weights_3 * (1-self.alpha-self.beta)

        return item_weights

In [ ]:
difflosshybridrecommender = DifferentLossScoresHybridRecommender(URM_train, SLIM, iALS, pureSVD)

# CHOOSE PROPER ALPHA AND BETA TO ASSIGN AS WEIGHT TO THE RECOMMENDERS
for alpha in list(np.arange(0.75, 0.9, 0.01)):
    for beta in list(np.arange(0.01, 1-alpha, 0.01)):
        difflosshybridrecommender.fit(norm = -np.inf, alpha = alpha, beta = beta)

        result_df, _ = evaluator_validation.evaluateRecommender(difflosshybridrecommender)
        print("Norm = -inf, Alpha = {}, Beta = {}, Result: {} ".format(alpha, beta, result_df.loc[10]["MAP"])) 

'''for norm in [1, 2, np.inf, -np.inf]:
    for alpha in list(np.arange(0.5, 1, 0.05)):
        for beta in list(np.arange(0.05, 1-alpha, 0.05)):
            difflosshybridrecommender.fit(norm, alpha, beta)
            
            result_df, _ = evaluator_validation.evaluateRecommender(difflosshybridrecommender)
            print("Norm = {}, Alpha = {}, Beta = {}, Result: {} ".format(norm, alpha, beta, result_df.loc[10]["MAP"])) 
'''
